# Installation

This library is not tracked by the Julia central package repository for the moment.

To install it, use `git clone` to put in in a convenient for you directory, go to that directory, open the Julia interpreter and type `] dev .` (`]` lets you enter the package manager prompt, `dev` means "install from directory, in a way that continues let me edit the code in this directory" and `.` means "current directory").

If you want to use the notebook interface, also perform `add IJulia` from that same package manager prompt and then from the main Julia prompt do `using IJulia; notebook(dir=".")`. If you want to use multithreaded execution use `ENV["JULIA_NUM_THREADS"] = 8; using IJulia; notebook(dir=".")`.

If you want to edit the code and have all your notebooks automatically use the new code without restarting them, you can also install `Revise` with `add Revise` in the package manager prompt. For benchmarking `add BenchmarkTools`.

### Some naming conventions might change before this is pushed to the Julia package repository!!!

In [1]:
#using Revise
using SimpleClifford

# Pauli Operators

They are stored in memory as a phase (a single byte where `0x0,0x1,0x2,0x3` corresponds to $1,i,-1,-i$) and two bit-arrays, for X and for Z components.

In [2]:
# Create them with P-strings
P"-iXZ"

-iXZ

In [3]:
# Or by specifying phase and X/Z components
PauliOperator(0x0,Bool[0,1,0],Bool[0,0,1])

+ _XZ

In [4]:
# Both underscore and I can be used for identity
P"I_XYZ"

+ __XYZ

In [5]:
# Available operations
-1im*P"X", P"X" * P"Z", P"X" ⊗ P"Z"

(-iX, -iY, + XZ)

In [6]:
# Commutativity checks
comm(P"X",P"Z"), comm(P"XX",P"ZZ")

(0x01, 0x00)

In [7]:
# Calculation of only the phase of a product
prodphase(P"X", P"Z"), prodphase(P"X", P"iZ"), prodphase(P"X",P"Y")

(0x03, 0x00, 0x01)

In [8]:
# Representation in memory
p = P"-IXYZ"
p.nqbits, p.phase, p.xz

(4, 0x02, UInt64[0x0000000000000006, 0x000000000000000c])

In [9]:
# Convenience attributes
p.xbit, p.zbit

(Bool[0, 1, 1, 0], Bool[0, 0, 1, 1])

In [10]:
# TODO No indexing is implemented for the moment
# p[1] # MethodError
# p[1] = ... # What should the API even look like?

# Stabilizers

They are stored in memory as a phase list and a bit-matrix for X and Z components.

In [11]:
# Create them with S-strings
S"-XX
  +ZZ"

- XX
+ ZZ

In [12]:
# Create them with list of Pauli operators
Stabilizer([P"-XX",P"+ZZ"])

- XX
+ ZZ

In [13]:
# Create them with list of phases and an X and Z matrices
Stabilizer([0x2, 0x0],
    Bool[1 1;
         0 0],
    Bool[0 0;
         1 1])

- XX
+ ZZ

In [14]:
# Available operations
S"-XX" ⊕ S"ZZ"

- XX__
+ __ZZ

In [15]:
# Indexing by an int gives you the corresponding Pauli operator
s = S"-XXX
      +ZZX
      +III"
s[2]

+ ZZX

In [16]:
# Indexing by a range gives you a subset of the stabilizer
s[1:2] # This is useful if you want to start with a large all-identity stabilizer and build it up bit by bit

- XXX
+ ZZX

In [17]:
# Assignment is also possible
s[1] = P"+YYX"
s

+ YYX
+ ZZX
+ ___

In [18]:
# Consistency at creation is not verified so nonsensical stabilizers can be created, both in terms of content and shape.
S"iX
  +Z"

+iX
+ Z

In [19]:
# Representation in memory
s = S"-XXX
      +ZZI
      -IZZ"
s.phases, s.nqbits, s.xzs

(UInt8[0x02, 0x00, 0x02], 3, UInt64[0x0000000000000007 0x0000000000000000; 0x0000000000000000 0x0000000000000003; 0x0000000000000000 0x0000000000000006])

In [20]:
# Convenience attributes
# TODO

# Canonicalization of Stabilizers

In [21]:
# Canonicalization (akin to Gaussian elimination over F(2,2)) is implemented.
s = S"-XXX
      +ZZX
      +III"
canonicalize!(s)

+ YY_
+ ZZX
+ ___

In [22]:
# If phases are inconsequential, the operations can be faster by not tracking and updating them.
s = S"-XXX
      +ZZX
      +III"
canonicalize!(s; phases=false)

- YY_
+ ZZX
+ ___

In [23]:
# These operations are in place (as customarily signified by "!")
s = S"-XXX
      +ZZX
      +III"
canonicalize!(s; phases=false)
s

- YY_
+ ZZX
+ ___

# Projective measurements

In [24]:
# This happens to be a GHZ state
s = S"-XXX
      +ZZI
      -IZZ";

In [25]:
# It returns the new stabilizer, the index where the anticommutation was detected,
# and the result of the projection (`nothing` being an undetermined result)
project!(copy(s), P"ZII")

(+ Z__
+ ZZ_
- _ZZ, 1, nothing)

In [26]:
# Here there are changes to the state (stabilizer), no anticommuting terms (the index is zero),
# and the result is perfectly determined (-1, or in our convention to represent the phase, 0x2).
project!(copy(s), P"-ZZI")

(- XXX
- Z_Z
- _ZZ, 0, 0x02)

In [27]:
# When the projection is consistent with the stabilizer (i.e. the mesurement result is not `nothing`),
# this would trigger an expensive canonicalization procedure in order to calculate the measurement 
# result. If all you want to know is whether the projection is consistent with the stabilizer, but
# you do not care about the measurement result, you can skip the canonicalization and calculation of
# the result.
project!(copy(s), P"-ZZI", keep_result=false)

(- XXX
+ ZZ_
- _ZZ, 0, nothing)

In [28]:
# Lastly, in either case, you can skip the calculation of the phases as well, if they are unimportant.
project!(copy(s), P"ZZI", phases=false)

(- XXX
+ Z_Z
- _ZZ, 0, 0x00)

# Generating a Pauli operator with Stabilizer generators

## i.e. checking for independence

In [29]:
# It requires the stabilizer to be already canonicalized
s = canonicalize!(s)

- XXX
- Z_Z
- _ZZ

In [30]:
# It modifies the Pauli operator in place, reducing it to identity if possible.
# The leftover phase is present to indicate if the phase itself could not have been canceled.
# The list of indices specifies which rows of the stabilizer were used to generated the
# desired Pauli operator.
generate!(P"XYY", s)

(- ___, [1, 3])

In [31]:
# Phases can be neglected, for higher performance.
generate!(P"XYY", s, phases=false)

(+ ___, [1, 3])

In [32]:
# If the Pauli operator can not be generated by the stabilizer, `nothing` value is returned.
generate!(P"ZZZ", s), generate!(P"XZX", s), generate!(P"YYY", s)

(nothing, nothing, nothing)

# Clifford Operators

In [33]:
# Predefined
println(Hadamard)
println(Phase)
println(CNOT)
println(CliffordId)

X ⟼ + Z
Z ⟼ + X

X ⟼ + Y
Z ⟼ + Z

X_ ⟼ + XX
_X ⟼ + _X
Z_ ⟼ + Z_
_Z ⟼ + ZZ

X ⟼ + X
Z ⟼ + Z



In [34]:
# Tensor products # IT IS SLOW
Hadamard ⊗ Phase

X_ ⟼ + Z_
_X ⟼ + _Y
Z_ ⟼ + X_
_Z ⟼ + _Z


In [35]:
# Chaining # IT IS SLOW
Hadamard * Phase

X ⟼ - Y
Z ⟼ + X


In [36]:
# Permuting qubits # IT IS SLOW
permute(CNOT, [2,1])

X_ ⟼ + X_
_X ⟼ + XX
Z_ ⟼ + ZZ
_Z ⟼ + _Z


In [37]:
# You can create custom Clifford operators with C-strings
C"-ZZ
  +_Z
  -X_
  +XX"

X_ ⟼ - ZZ
_X ⟼ + _Z
Z_ ⟼ - X_
_Z ⟼ + XX


In [38]:
# Or as a list of Pauli operators
CliffordOperator([P"-ZZ", P"_Z", P"-X_", P"XX"])

X_ ⟼ - ZZ
_X ⟼ + _Z
Z_ ⟼ - X_
_Z ⟼ + XX


In [39]:
# TODO: creating them by using a Stabilizer or by using boolean matrices

In [40]:
# Applying them to Stabilizers.
CNOT * S"X_"

+ XX

In [41]:
# Applying them in-place for higher performance.
# Neglecting phases for higher performance can be done with `phases=false`.
s = S"X_"
apply!(s,CNOT)
s

+ XX

In [42]:
# TODO / incomplete: Small Clifford operators can be applied to large stabilizers by specifying the qubit indices.

In [43]:
# Pauli operators act as Clifford operators too (but they are rather boring, as they only change signs).
P"XII" * S"ZXX"

- ZXX